In [7]:
from sickle import Sickle
import xml.etree.ElementTree as ET
import pprint
import pandas as pd

sickle = Sickle('https://texashistory.unt.edu/oai')
#records = sickle.ListRecords(metadataPrefix='oai_dc')

In [14]:
# other metadata formats:
#untl - 2159
# oai_dc - 2157
# untl_raw - 2159
# untl_dpla - 2159
# rdf - 2159

records = sickle.ListRecords(metadataPrefix='oai_dc', set='collection:SJCC')

In [10]:
#records.next()

<Record info:ark/67531/metapth1634318>

In [50]:
# count of records
print(sum(1 for e in records))

2159


In [15]:
rec_count = 0
relation_count = 0
data = []
for record in records:
    #print(record.metadata)
    rec_count += 1
    # get ARC
    rec_identifier = record.header.identifier
    arc = rec_identifier.split(':')[1]
    #print(arc)
    metadata = record.metadata
    title = record.metadata.get('title')
    #creator = record.metadata.get('creator')
    subject = record.metadata.get('subject')
    description = record.metadata.get('description')
    date = record.metadata.get('date')
    type = record.metadata.get('type')
    format = record.metadata.get('format')
    identifier = record.metadata.get('identifier')
    #coverage = record.metadata.get('coverage')
    relation = record.metadata.get('relation')
    xml_str = ET.tostring(record.xml, encoding='unicode')
    
    if relation:
        relation_count += 1
        print(relation)
        relation_string = relation[0]
        #pprint.pprint(record.header)
        if 'same individual' in relation_string:
            relation_type = 'specimen'
        elif 'population' in relation_string:
            relation_type = 'population'
        else:
            relation_type = 'undefined'
        #print(relation_string, relation_type)
    else:
        print('no relation')
        relation_type = None
    print('relation_type:', relation_type)

    # add row
    # intermediate detail
    # data.append({'rec_identifier': rec_identifier, 'arc': arc, 'metadata': metadata, 'title': title, 'subject': subject, 'description': description, 'date': date, 'type': type, 'format': format, 'identifier': identifier, 'relation': relation, 'xml_str': xml_str})
    # simple detail
    data.append({'rec_identifier': rec_identifier, 'arc': arc, 'title': title, 'subject': subject, 'description': description, 'date': date,  'relation': relation, 'relation_type': relation_type })



# convert list to dataframe
df = pd.DataFrame(data)

        
print('rec_count', rec_count)
print('relation_count', relation_count)

no relation
relation_type: None
no relation
relation_type: None
no relation
relation_type: None
no relation
relation_type: None
no relation
relation_type: None
no relation
relation_type: None
no relation
relation_type: None
no relation
relation_type: None
no relation
relation_type: None
no relation
relation_type: None
no relation
relation_type: None
no relation
relation_type: None
no relation
relation_type: None
no relation
relation_type: None
no relation
relation_type: None
no relation
relation_type: None
no relation
relation_type: None
no relation
relation_type: None
no relation
relation_type: None
no relation
relation_type: None
no relation
relation_type: None
no relation
relation_type: None
no relation
relation_type: None
no relation
relation_type: None
no relation
relation_type: None
no relation
relation_type: None
no relation
relation_type: None
no relation
relation_type: None
no relation
relation_type: None
no relation
relation_type: None
no relation
relation_type: None
no relat

In [16]:
df.to_csv('sjcc_all.csv')

In [44]:
print(one_rec)

<record xmlns="http://www.openarchives.org/OAI/2.0/" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"><header><identifier>info:ark/67531/metapth1634321</identifier><datestamp>2024-08-26T12:35:29Z</datestamp><setSpec>partner:BRIT</setSpec><setSpec>collection:SJCC</setSpec><setSpec>access_rights:public</setSpec></header><metadata><oai_dc:dc xmlns:oai_dc="http://www.openarchives.org/OAI/2.0/oai_dc/" xmlns:dc="http://purl.org/dc/elements/1.1/" xsi:schemaLocation="http://purl.org/dc/elements/1.1/ http://www.openarchives.org/OAI/2.0/oai_dc.xsd"><dc:title>[Emu in Australia]</dc:title><dc:creator>Carlquist, Sherwin John, 1930-2021</dc:creator><dc:subject>Scientific expeditions</dc:subject><dc:subject>Landscape and Nature - Wildlife</dc:subject><dc:subject>color slides</dc:subject><dc:subject>animals</dc:subject><dc:subject>fieldwork</dc:subject><dc:subject>field expeditions</dc:subject><dc:description>Photograph of an emu in Australia. The emu is brown with a dark head and is facing the l